In [ ]:
#Esta abordagem é filtragem colaborativa baseada em itens (item-based collaborative filtering)
!pip install pandas numpy scipy scikit-learn

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Configurar diretório e carregar dados

!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

dataset_dir = "ml-latest-small"

#Carregar avaliações
ratings = pd.read_csv(f"{dataset_dir}/ratings.csv")  #userId, movieId, rating, timestamp

#Carregar filmes
movies = pd.read_csv(f"{dataset_dir}/movies.csv")    #movieId, title, genres

print("Dados carregados!")
print(f"Total de avaliações: {len(ratings)}")
print(f"Total de filmes: {len(movies)}")

In [ ]:
#Mapear IDs para índices e criar matriz esparsa

#O objetivo aqui é transformar os IDs originais (que não são sequenciais)
#em índices contínuos (0, 1, 2, ...) — algo necessário para criar uma matriz.

#Cria dicionários de mapeamento de IDs para índices
#Cada linha em ratings representa uma avaliação feita por um usuário a um filme
#.unique() retorna um array NumPy contendo apenas os valores distintos de uma coluna, na ordem em que aparecem
#enumerate() transforma uma sequência em pares (índice, valor)
'''Se ratings.csv contiver:
   | userId | movieId | rating |
   | ------ | ------- | ------ |
   | 10     | 5       | 3.5    |
   | 42     | 2       | 4.0    |
   | 10     | 7       | 2.0    |
   Então user_mapper = {10: 0, 42: 1}, movie_mapper = {5: 0, 2: 1, 7: 2}
   Usuário 10 vira linha 0, Usuário 42 vira linha 1
   Filme 5 vira coluna 0, Filme 2 vira coluna 1, Filme 7 vira coluna 2
   Matriz esparsa ficará com shape (2, 3) — 2 usuários × 3 filmes.
'''

#Cria mapeamento de cada userId único para um índice sequencial
user_mapper = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
#Cria mapeamento de cada movieId único para um índice sequencial
movie_mapper = {mid: i for i, mid in enumerate(ratings['movieId'].unique())}

#Mapear IDs originais para índices
ratings['user_idx'] = ratings['userId'].map(user_mapper)
ratings['movie_idx'] = ratings['movieId'].map(movie_mapper)

#Criar matriz esparsa (usuários x filmes)
ratings_sparse = csr_matrix(
    (ratings['rating'], (ratings['user_idx'], ratings['movie_idx']))
)

'''Com o exemplo usado antes:
   user_mapper = {10: 0, 42: 1}
   movie_mapper = {5: 0, 2: 1, 7: 2}
   A matriz de usuários × filmes fica:        5     2     7
                                       U10 [ 3.5   0.0   2.0 ]
                                       U42 [ 0.0   4.0   0.0 ]
   Então a representação esparsa (armazenando só valores != 0) fica:
            (0,0)  3.5 --> user 10, movie 5
            (0,2)  2.0 --> user 10, movie 7
            (1,1)  4.0 --> user 42, movie 2
'''

#Mapeamento inverso (para retornar IDs originais)
movie_inv_mapper = {i: mid for mid, i in movie_mapper.items()}
user_inv_mapper = {i: uid for uid, i in user_mapper.items()}

print("Matriz esparsa criada!")
print(f"Shape da matriz: {ratings_sparse.shape}")

In [ ]:
#Calcular similaridade entre filmes

#.T = operador de transposição. ratings_sparse tem usuários nas linhas e filmes nas colunas
#Transposição é feita para ter filmes nas linhas (necessário para filme-filme similarity)
#dense_output=False mantém saída esparsa.
movie_similarity = cosine_similarity(ratings_sparse.T, dense_output=False)

print("Matriz de similaridade entre filmes calculada!")

In [10]:
#Função para recomendar filmes similares
#Retorna os top_n filmes mais similares a um movie_id dado
def recommend_movies(movie_id, top_n=5):
    #Obter índice do filme
    if movie_id not in movie_mapper:
        print(f"Filme {movie_id} não encontrado.")
        return []

    #Converte movie_id original para o índice interno da matriz
    movie_idx = movie_mapper[movie_id]

    #Extrai a linha correspondente do filme na matriz de similaridade
    #(cada valor indica o quão parecido é com outro filme)
    sim_scores = movie_similarity[movie_idx].toarray().flatten()

    #Cria uma Series (vetor do pandas) com os scores de similaridade, movie_idx como índice
    sim_series = pd.Series(sim_scores, index=range(len(sim_scores)))

    #Seleciona os índices dos filmes mais similares, em ordem decrescente
    #iloc[1:] ignora o próprio filme (similaridade = 1 consigo mesmo)
    top_indices = sim_series.sort_values(ascending=False).iloc[1:top_n+1].index

    #Monta a lista de recomendações (título e valor da similaridade)
    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]                                   #Converte índice interno de volta para movieId original
        title = movies.loc[movies.movieId == mid, 'title'].values[0]  #Busca título do filme
        score = sim_series[idx]                                       #Similaridade calculada
        recommendations.append((title, score))

    return recommendations

In [ ]:
#Testar recomendações para um filme

#Exemplo: movieId = 1 (Toy Story 1995)
movie_id = 1
top_movies = recommend_movies(movie_id, top_n=5)

print(f"Top 5 filmes similares a '{movies.loc[movies.movieId==movie_id,'title'].values[0]}':")
for title, score in top_movies:
    print(f"{title} — similaridade {score:.3f}")

In [12]:
#Função para recomendar para um usuário
#Retorna recomendações para um usuário baseado em ratings passados e similaridade filme-filme
def recommend_for_user(user_id, top_n=5):
    #Obter índice do usuário
    if user_id not in user_mapper:
        print(f"Usuário {user_id} não encontrado.")
        return []

    #Converte user_id original para o índice interno da matriz
    user_idx = user_mapper[user_id]

    #Filmes avaliados pelo usuário (linha do usuário na matriz)
    #Converte a linha esparsa para array e achata para vetor 1D
    user_ratings = ratings_sparse[user_idx, :].toarray().flatten()

    #Pega os índices dos filmes avaliados (notas > 0)
    rated_indices = np.where(user_ratings > 0)[0]

    #Vetor inicial para acumular scores de recomendação
    #(um valor predito para cada filme)
    scores = np.zeros(ratings_sparse.shape[1])

    '''Para cada filme que o usuário avaliou:
       - pega as similaridades desse filme com todos os outros
       - soma uma contribuição proporcional à nota que o usuário deu'''
    for idx in rated_indices:
        sim_scores = movie_similarity[idx].toarray().flatten()
        scores += sim_scores * user_ratings[idx]

    #Zera os scores dos filmes já avaliados para não recomendar algo que o usuário já viu
    scores[rated_indices] = 0

    #Seleciona os índices dos filmes com maiores scores preditos
    top_indices = np.argsort(scores)[::-1][:top_n]

    #Monta a lista final de recomendações com títulos e pontuações
    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]   #Converte índice interno para movieId original
        title = movies.loc[movies.movieId == mid, 'title'].values[0]
        recommendations.append((title, scores[idx]))

    return recommendations

In [ ]:
#Testar recomendações para um usuário

user_id = 26
top_user_movies = recommend_for_user(user_id, top_n=5)

print(f"Top 5 recomendações para o usuário {user_id}:")
for title, score in top_user_movies:
    print(f"{title} — score {score:.3f}")